In [9]:
import sys

from src.data_loader import load_kaggle_data
import os
os.chdir("/home/rana-helal/PycharmProjects/fake_news_classifier")

df_kaggle = load_kaggle_data()


In [11]:
!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
from tqdm import tqdm
tqdm.pandas()


In [14]:
from src.preprocessing import remove_duplicates_and_missing, clean_for_ml

# Apply text cleaning for ML models

df_cleaned = remove_duplicates_and_missing(df_kaggle)


In [ ]:

df_cleaned['clean_text'] = df_kaggle['text'].progress_apply(clean_for_ml)


  1%|          | 516/44898 [00:20<30:37, 24.15it/s]

In [ ]:
from src.utils import save_cleaned_data

save_cleaned_data(df_cleaned, "kaggle_clean_ml.csv")